In [1]:
import jax
import jax.numpy as jnp
from jax.config import config;
config.update("jax_enable_x64", True)

from scipy.integrate import quad

import yaml

import buW2

In [2]:
with open("buW2/parameters.yaml", "r") as input_file:
    pars = yaml.load(input_file, Loader = yaml.FullLoader)

pars = pars["Parameters"]

with open("buW2/wi_R2_parameters.yaml", "r") as input_file:
    wi_R2_pars = yaml.load(input_file, Loader = yaml.FullLoader)

wi_R2_pars = jnp.array(wi_R2_pars["pars"])

In [3]:
w0 = buW2.w0(buW_pars = pars)

In [4]:
wv1 = buW2.wV1(buW_pars = pars)
wr1 = buW2.wR1(buW_pars = pars)
wr1_subt = buW2.wR1Subt(buW_pars = pars)
wr1_surf = buW2.wR1Surf(buW_pars = pars)

In [5]:
wv2 = buW2.wV2(buW_pars = pars)
wr2 = buW2.wR2(buW_pars = pars)
wr2_subt = buW2.wR2Subt(buW_pars = pars)
wr2_surf = buW2.wR2Surf(buW_pars = pars)

In [6]:
eps = 1e-3

q2h_vec = jnp.linspace(0.15, 0.35, 50)

q2h_tensor = jnp.tile(q2h_vec[None], reps = (25, 1))
q0h_tensor = jnp.linspace(jnp.sqrt(q2h_vec) + eps, ((1 + q2h_vec) / 2) - eps, 25)

q0h_min = 0.10

# LO

In [7]:
w1_0_tensor = w0("W1", q2h_tensor)
w2_0_tensor = w0("W2", q2h_tensor)
w3_0_tensor = w0("W3", q2h_tensor)

# NLO

In [8]:
w1_v1_tensor = wv1("W1", q2h_tensor)
w2_v1_tensor = wv1("W2", q2h_tensor)
w3_v1_tensor = wv1("W3", q2h_tensor)

In [9]:
w1_r1_tensor = wr1("W1", q0h_tensor, q2h_tensor)
w2_r1_tensor = wr1("W2", q0h_tensor, q2h_tensor)
w3_r1_tensor = wr1("W3", q0h_tensor, q2h_tensor)

In [10]:
w1_r1_subt_tensor = wr1_subt("W1", q0h_tensor, q2h_tensor)
w2_r1_subt_tensor = wr1_subt("W2", q0h_tensor, q2h_tensor)
w3_r1_subt_tensor = wr1_subt("W3", q0h_tensor, q2h_tensor)

In [11]:
w1_r1_surf_tensor = wr1_surf("W1", q2h_tensor, q0h_min)
w2_r1_surf_tensor = wr1_surf("W2", q2h_tensor, q0h_min)
w3_r1_surf_tensor = wr1_surf("W3", q2h_tensor, q0h_min)

# NNLO

In [12]:
w1_v2_tensor = wv2("W1", q2h_tensor)
w2_v2_tensor = wv2("W2", q2h_tensor)
w3_v2_tensor = wv2("W3", q2h_tensor)

In [13]:
w1_r2_tensor = wr2("W1", q0h_tensor, q2h_tensor, wi_R2_pars)
w2_r2_tensor = wr2("W2", q0h_tensor, q2h_tensor, wi_R2_pars)
w3_r2_tensor = wr2("W3", q0h_tensor, q2h_tensor, wi_R2_pars)

In [14]:
w1_r2_subt_tensor = wr2_subt("W1", q0h_tensor, q2h_tensor)
w2_r2_subt_tensor = wr2_subt("W2", q0h_tensor, q2h_tensor)
w3_r2_subt_tensor = wr2_subt("W3", q0h_tensor, q2h_tensor)

In [15]:
w1_r2_surf_tensor = wr2_surf("W1", q2h_tensor, q0h_min)
w2_r2_surf_tensor = wr2_surf("W2", q2h_tensor, q0h_min)
w3_r2_surf_tensor = wr2_surf("W3", q2h_tensor, q0h_min)

# Integrating a smooth $f(\hat{q}_0, \hat{q}^2)$ function against $W_i$

In [16]:
def f(q0h, q2h):
    return jnp.exp(-(q0h + q2h)**2)

In [17]:
def delta(q2h):
    q0h_end_point = (1 + q2h) / 2
    
    w1_v2 =  wv2("W1", q2h)
    
    return (1 / 2) * f(q0h_end_point, q2h) * w1_v2

In [18]:
@jax.jit
def integrand_real(q0h, q2h):
    w1_r2 =  wr2("W1", q0h, q2h, wi_R2_pars)
    w1_r2_subt = wr2_subt("W1", q0h, q2h)
    
    return f(q0h, q2h) * (w1_r2 + w1_r2_subt)

def real(q2h):
    q0h_end_point = (1 + q2h) / 2
    q0h_threshold = jnp.sqrt(q2h)
    
    integrand = lambda q0h: integrand_real(q0h, q2h)
    
    return quad(integrand, q0h_threshold, q0h_end_point)[0]

In [19]:
def surface(q2h):
    q0h_end_point = (1 + q2h) / 2
    q0h_threshold = jnp.sqrt(q2h)
    
    w1_r2_surf =  wr2_surf("W1", q2h, q0h_threshold)
    
    return f(q0h_end_point, q2h) * w1_r2_surf

In [20]:
def f_W1_integral(q2h):    
    return delta(q2h) + real(q2h) + surface(q2h)

In [21]:
f_W1_integral(0.25)

Array(-1.28986837, dtype=float64)